In [12]:
from keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import pandas


# パーセプトロンの数
N = 4  # 入力層の数
K = 20  # 隠れ層の数
M = 2  # 出力層の数


data = pandas.read_csv("std_race_data.csv")

# カラム指定の上列抽出
# この時点では
# rade_id, horse_age, weight...の行列
training_data = data.loc[:, ["race_id", "horse_age",
                             "weight", "hourse_weight", "weight_change", "single_odds", "multi_odds"]]
np_array = training_data.values

# mapにばらして、race_idごとに分けていく
# つまり
# mp[race_id_no1] = [
#   [1,2,3,4,5,6],
#   [1,2,3,4,5,6],
# ]
# みたいなデータが出来る
# ついでに正解データも作る
mp = {}
mp_answer = {}
for r in np_array:
    if r[0] in mp:
        mp[r[0]].append(r[1:5].tolist())
        mp_answer[r[0]].append(r[5:7].tolist())
    else:
        mp[r[0]] = []
        mp_answer[r[0]] = []
        mp[r[0]].append(r[1:5].tolist())
        print(r)
        mp_answer[r[0]].append(r[5:7].tolist())

tmp_data = []
tmp_answer = []
for key in mp.keys():
    tmp_data.append(mp[key])
    tmp_answer.append(mp_answer[key])

# ちょい怠いので一旦16頭建てのレース以外排除する
training_data = []
answer_data = []
for index in range(0, len(tmp_data)):
    if len(tmp_data[index]) == 16:
        training_data.append(tmp_data[index])
        answer_data.append(tmp_answer[index])

trainingList = training_data
answerList = answer_data
# d = pandas.DataFrame(training_data)
# a = pandas.DataFrame(answer_data)
# trainingList = d.values
# answerList = a.values


[ 2.02203020e+11 -3.71234895e-01 -1.82290829e-01  4.67186637e-01
  1.01918354e+00 -6.87992098e-01 -6.19412766e-01]
[ 2.02203020e+11 -3.71234895e-01 -1.82290829e-01 -2.05171200e-01
 -1.81782672e+00 -5.44670831e-01 -4.64932963e-01]
[ 2.02203020e+11 -1.15025484e+00 -1.19261655e+00 -2.72406984e-01
  7.35134518e-02 -6.64302632e-01 -5.90265633e-01]
[ 2.02203020e+11 -1.15025484e+00 -1.82290829e-01 -4.74114335e-01
  7.35134518e-02 -6.57195793e-01 -5.93180347e-01]
[ 2.02203020e+11 -3.71234895e-01  8.28034889e-01 -4.74114335e-01
  3.88736813e-01 -6.83254205e-01 -6.10668626e-01]
[ 2.02203020e+11  1.18680499e+00  1.33319775e+00  8.70601339e-01
 -2.41709910e-01 -6.06263442e-01 -5.26141942e-01]
[ 2.02203020e+11 -3.71234895e-01 -1.19261655e+00 -1.95330158e+00
  1.01918354e+00 -6.64302632e-01 -5.84436207e-01]
[ 2.02203020e+11  4.07785048e-01  1.33319775e+00  1.07230869e+00
  3.88736813e-01 -6.69040526e-01 -5.90265633e-01]
[ 2.02203020e+11  1.18680499e+00 -1.82290829e-01 -7.06996327e-02
  7.35134518e-0

In [13]:

# 入力データ

# 学習データ(名前,騎手,会場など)
# trainingList = np.array([
#     [
#         [-0.6879920980515413, -0.6194127660107818, 0.1234, -0.1234],
#         [-0.6358752736489328, -0.5028242358046408, 0.1234, -0.1234]
#     ],
#     [
#         [-0.6879920980515413, -0.6194127660107818, 0.1234, -0.1234],
#         [-0.6358752736489328, -0.5028242358046408, 0.1234, -0.1234]
#     ]
# ])

# 教師データ(単勝オッズ、複勝オッズ)
# answerList = np.array([
#     [
#         [0.9, 0.3],
#         [0.3, 0.1]
#     ],
#     [
#         [0.9, 0.3],
#         [0.3, 0.1]
#     ]
# ])

# テストデータ
# データ構造は学習データと同じ。学習済モデルの精度を上げるために必要
# testingList = np.array([
#     [
#         [-0.6879920980515413, -0.6194127660107818, 0.1234, -0.1234],
#         [-0.6358752736489328, -0.5028242358046408, 0.1234, -0.1234]
#     ],
#     [
#         [-0.6879920980515413, -0.6194127660107818, 0.1234, -0.1234],
#         [-0.6358752736489328, -0.5028242358046408, 0.1234, -0.1234]
#     ]
# ])


# 3層ニューラルネットワークの構築
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(16, 4)))
model.add(tf.keras.layers.Dense(K, activation='tanh'))
model.add(tf.keras.layers.Dense(M, activation='softmax'))

#  fitするためにコンパイルする
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
)

# モデルの概要表示
model.summary()

# 固定回数（データセットの反復）の試行でモデルを学習
model.fit(trainingList, answerList, None, 256)

# 出力 ※ 戻り値は numpy 配列
y = model.predict(testingList)

print('入力')
tf.print(testingList)
print('\n出力')
print(y)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16, 20)            100       
                                                                 
 dense_7 (Dense)             (None, 16, 2)             42        
                                                                 
Total params: 142
Trainable params: 142
Non-trainable params: 0
_________________________________________________________________
Epoch 1/256
2/2 [==============================] - 0s 4ms/step - loss: 0.7856
Epoch 2/256
2/2 [==============================] - 0s 4ms/step - loss: 0.7780
Epoch 3/256
2/2 [==============================] - 0s 4ms/step - loss: 0.7708
Epoch 4/256
2/2 [==============================] - 0s 5ms/step - loss: 0.7643
Epoch 5/256
2/2 [==============================] - 0s 4ms/step - loss: 0.7579
Epoch 6/256
2/2 [==============================] - 0s 4ms/step

ValueError: in user code:

    File "/home/tomoyan/horse-fire/env/lib/python3.8/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/home/tomoyan/horse-fire/env/lib/python3.8/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/tomoyan/horse-fire/env/lib/python3.8/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/home/tomoyan/horse-fire/env/lib/python3.8/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/home/tomoyan/horse-fire/env/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/tomoyan/horse-fire/env/lib/python3.8/site-packages/keras/engine/input_spec.py", li